In [1]:
!cp /content/drive/MyDrive/data.mail.xlsx data.mail.xlsx

In [1]:
!pip install accelerate>=0.21.0

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import sys
import numpy as np
import pickle
import re
from nltk.stem import SnowballStemmer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
nltk.download("stopwords")
tqdm.pandas()

[nltk_data] Downloading package stopwords to /home/datadisk/jupyter-
[nltk_data]     admin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
excel_file = "data.mail.xlsx"
all_sheets = pd.read_excel(excel_file, sheet_name=None)
sheets = all_sheets.keys()
for sheet_name in sheets:
    sheet = pd.read_excel(excel_file, sheet_name=sheet_name)
    sheet.to_csv("%s.csv" % sheet_name, index=False)

In [5]:
stemmer = SnowballStemmer("russian", ignore_stopwords=True)
tokenizer = RegexpTokenizer(r'\w+')
stop = set(stopwords.words('russian'))

def clear_text(text):
    return ' '.join(stemmer.stem(x) for x in tokenizer.tokenize(text.lower()) if x not in stop)

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/datadisk/jupyter-
[nltk_data]     admin/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

['лемматизирова', 'форм', 'слов', 'лист', 'эт', 'лист']


In [9]:
df = pd.read_csv("deti.mail.csv")
df

,URL,Title,Description,Article Text,Keywords
0,https://deti.mail.ru/article/kostyumy-na-novyj...,Костюмы на Новый год для детей своими руками,Сделать новогодний наряд для сына или дочки св...,Костюм Снежинки В качестве основы для костюма ...,Дети 3-7 лет
1,https://deti.mail.ru/article/esli-rebenok-veri...,Как создают детское новогоднее шоу: интервью с...,"Режиссер, сценарист и главный новогодний волше...",Если ребенок верит в сказку хотя бы до десяти ...,Семья
2,https://deti.mail.ru/article/gid-po-detskim-no...,30 крутых детских новогодних подарков на любой...,"Думаете о том, что подарить ребенку на Новый г...",Подарки до 1 000 рублей Наборы для создания сн...,Семья
3,https://deti.mail.ru/article/chto-delat-esli-s...,"Что делать, если с ГВ не сложилось: 4 совета","Лучшая еда для младенца – это, конечно, мамино...",Чаще практикуйте контакт кожа к коже Период но...,1 месяц - 6 месяцев
4,https://deti.mail.ru/article/novyj-god-blizko-...,Новый год близко: как звезды с детьми наряжают...,Главный атрибут праздника уже появился в домах...,"На календаре почти середина декабря, и знамени...",Семья
...,...,...,...,...,...
5962,https://deti.mail.ru/article/uslysh/,"""Услышь себя"". Тест для родителей",Часто ли родители слышат самих себя? Какова сп...,От вас требуется лишь максимально честно отмет...,Дети 3-7 лет
5963,https://deti.mail.ru/article/disp/,Что такое диспансеризация и зачем она нужна?,Диспансеризация - это метод медицинского обслу...,Самое главное при диспансеризации во время бер...,Беременность
5964,https://deti.mail.ru/article/lor/,Консультация у отоларинголога в период беремен...,У будущих мам часто возникает вопрос - с какой...,В настоящее время у 65-70% обследованных берем...,Беременность
5965,https://deti.mail.ru/article/test__nelepicy__d...,"Тест ""Нелепицы"". Для детей 3-6 лет",Образ мышления дошкольников представлен образа...,Для тестирования вам понадобится заранее подго...,Дети 3-7 лет


In [8]:
df["text"] = df["Article Text"].progress_apply(clear_text)

100%|██████████| 5967/5967 [03:06<00:00, 31.94it/s]


In [10]:
df["labels"], indexer = df["Keywords"].factorize()

In [18]:
text_clf = Pipeline([
    ('tfidf', CofVectorizer()),
    ('clf', SGDClassifier(loss='hinge')),
])

In [13]:
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

nltk.download('punkt')

# Пример данных
texts = df["Article Text"]
labels = df["labels"]  # 1 - позитивный, 0 - негативный

# Токенизация
tokens = [word_tokenize(text) for text in texts]

# Создание BoW модели
vectorizer = CountVectorizer()
bow = vectorizer.fit_transform([' '.join(token) for token in tokens])

# Разделение данных на обучающую и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(bow, labels, test_size=0.1)

# Обучение классификатора
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

# Оценка классификатора
predictions = classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))

[nltk_data] Downloading package punkt to /home/datadisk/jupyter-
[nltk_data]     admin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Accuracy: 0.48408710217755446


In [14]:
predictions = classifier.predict(X_test)
print("f1: ", f1_score(predictions, y_test, average="macro"))

f1:  0.23753053552473347


In [19]:
train_x, test_x, train_y, test_y = train_test_split(df["text"], df["labels"], test_size=0.1)

###Test

In [33]:
text_clf = Pipeline([
    ('tfidf', CountVectorizer()),
    ('clf', SGDClassifier(loss='hinge')),
])

In [34]:
text_clf.fit(train_x, train_y)

Pipeline(steps=[('tfidf', CountVectorizer()), ('clf', SGDClassifier())])

In [35]:
pred = text_clf.predict(test_x)

In [39]:
f1_score(pred, test_y, average="macro")

0.4760804087899595

In [ ]:
import re
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer, roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB

In [ ]:
!pip install gradio
!pip install pandas
!pip install langchain
!pip install openai
!pip install langchain_community
!pip install langchain_openai

In [ ]:
import gradio as gr
import pandas as pd

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from langchain_core.output_parsers import JsonOutputParser

base_url = "http://saiga.vkcloud.eazify.net:8000/v1"
# base_url = "http://hackllm.vkcloud.eazify.net:8000/v1"
# base_url = "http://shwars.vkcloud.eazify.net:8000/v1"

chat = ChatOpenAI(api_key="<key>",
                model = "tgi",
                openai_api_base = base_url,
                temperature=0.2)

instruct = """
Прочитай список ключевых слов статьи:
Твой сотрудник, говорит, что эта статья категории "", но он может ошибаться. Напиши только категорию статьи в именительном падеже."
"""

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Ты редактор сайта новостей, твоя задача проверять корректность категории новостей по ключевым словам. На сайте есть категории: 'Общество', 'Экономика', 'Кино и телевидение', 'Люди','Наука и технологии', 'Транспорт', 'Погода', 'Рецепты', 'Мода', 'Дети', 'Дом', 'Питомцы', 'Здоровье'. Выбирать можно только их. "),
        ("human", instruct),
    ]
)


def process(text):
    res = chat.invoke(chat_template.format_messages(review=text))
    try:
        js = parser.invoke(res)
    except:
        js = { "positive" : [] , "negative" : [] }

    mx = max(len(js['positive']),len(js['negative']))
    while len(js['positive'])<mx:
        js['positive'].append('')
    while len(js['negative'])<mx:
        js['negative'].append('')

    return pd.DataFrame(js)

demo = gr.Interface(
    fn=process,
    inputs=["text"],
    outputs=["dataframe"],
)

demo.launch(server_name="0.0.0.0",server_port=8000)

In [ ]:
from gradio_client import Client

req = """Ты редактор сайта новостей. На сайте есть категории: 'Общество', 'Экономика', 'Кино и телевидение', 
'Люди','Наука и технологии', 'Транспорт', 'Погода', 'Рецепты', 'Мода', 'Дети', 'Дом', 'Питомцы', 'Здоровье'.
Выбирать можно только их.
Прочитай список ключевых слов статьи: последний, минута, отплытие, задокументированный, путешественник.
Твой сотрудник, говорит, что эта статья категории "Общество", но он может ошибаться. Напиши только категорию статьи в именительном падеже."""

client = Client("https://ilyagusev-saiga2-13b-gguf.hf.space/--replicas/b1eq7/")
result = client.predict(
    [[req ,None]],  # Tuple[str | Dict(file: filepath, alt_text: str | None) | None, str | Dict(file: filepath, alt_text: str | None) | None]  in 'Диалог' Chatbot component
    "Hello!!",  # str  in 'Системный промпт' Textbox component
    0,  # float (numeric value between 0.0 and 1.0) in 'Top-p' Slider component
    10,  # float (numeric value between 10 and 100) in 'Top-k' Slider component
    0,  # float (numeric value between 0.0 and 2.0) in 'Температура' Slider component
    api_name="/bot"
)
print(result[-1][-1])


In [11]:
!pip install typing_extensions

Defaulting to user installation because normal site-packages is not writeable


In [15]:
import gradio_client as gr
import pandas as pd

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from langchain_core.output_parsers import JsonOutputParser

base_url = "http://saiga.vkcloud.eazify.net:8000/v1"
# base_url = "http://hackllm.vkcloud.eazify.net:8000/v1"
# base_url = "http://shwars.vkcloud.eazify.net:8000/v1"

chat = ChatOpenAI(api_key="<key>",
                model="tgi",
                openai_api_base=base_url,
                temperature=0.2)

instruct = """
Прочитай приведённый ниже в тройных обратных кавычках тексь и верни все ключевые слова 
в формате JSON следующего вида:
{{
  "keywors" : [<ключевые слова>],
}}
Текст: ```{review}```"
"""

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Ты редактор сайта со статьями, и твоя извлечь из текста ключевые слова."),
        ("human", instruct),
    ]
)

parser = JsonOutputParser()

def process(text):
    res = chat.invoke(chat_template.format_messages(review=text))
    try:
        js = parser.invoke(res)
    except:
        js = { "keywors": "" }

    return str(js.get("keywors", ""))


demo = gr.Interface(
    fn=process,
    inputs=["text"],
    outputs=["text"],
)

demo.launch(server_name="localhost", server_port=8000)

AttributeError: module 'gradio_client' has no attribute 'Interface'